In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [49]:

# 1. Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


In [35]:
df = pd.read_csv("/kaggle/input/dataset-renamed/dataset_renamed.csv")


In [36]:
print("First 5 rows:")
print(df.head())

First 5 rows:
    Latitude  Longitude    Rainfall  Temperature   Humidity  River Discharge  \
0  18.861663  78.835584  218.999493    34.144337  43.912963      4236.182888   
1  25.361096  85.610733  198.984191    21.569354  34.453690      3683.208933   
2  25.432335  90.125517  212.870315    35.607657  60.765855      1867.752445   
3  14.157834  77.647041   95.933187    36.564520  93.939027      3138.637678   
4  23.217937  74.963672  291.654891    18.082377  48.357780      3994.697746   

   Water Level    Elevation  Land Cover Soil Type  Population Density  \
0     7.415552   377.465434  Water Body      Clay         7276.742184   
1     2.891787  2512.277800      Desert     Sandy         6163.069701   
2     2.027741  7287.016909      Forest      Silt         6782.941225   
3     5.421321   543.271851      Desert      Silt         4965.927515   
4     0.545035  6919.070203  Water Body      Loam         7144.881605   

   Infrastructure  Historical Floods  Flood Occurred  \
0         

In [37]:
df = df.drop('geometry', axis=1)

In [38]:
print("First 5 rows:")
print(df.head())

First 5 rows:
    Latitude  Longitude    Rainfall  Temperature   Humidity  River Discharge  \
0  18.861663  78.835584  218.999493    34.144337  43.912963      4236.182888   
1  25.361096  85.610733  198.984191    21.569354  34.453690      3683.208933   
2  25.432335  90.125517  212.870315    35.607657  60.765855      1867.752445   
3  14.157834  77.647041   95.933187    36.564520  93.939027      3138.637678   
4  23.217937  74.963672  291.654891    18.082377  48.357780      3994.697746   

   Water Level    Elevation  Land Cover Soil Type  Population Density  \
0     7.415552   377.465434  Water Body      Clay         7276.742184   
1     2.891787  2512.277800      Desert     Sandy         6163.069701   
2     2.027741  7287.016909      Forest      Silt         6782.941225   
3     5.421321   543.271851      Desert      Silt         4965.927515   
4     0.545035  6919.070203  Water Body      Loam         7144.881605   

   Infrastructure  Historical Floods  Flood Occurred           Sta

In [39]:
le_land = LabelEncoder()
le_soil = LabelEncoder()

df['Land Cover'] = le_land.fit_transform(df['Land Cover'])
df['Soil Type'] = le_soil.fit_transform(df['Soil Type'])

In [40]:
X = df.drop('Flood Occurred', axis=1)
y = df['Flood Occurred']

# Apply OneHotEncoding to State and District
categorical_features = ['State', 'District']
onehot = OneHotEncoder(drop='first', sparse=False)  # drop='first' to avoid dummy variable trap
ct = ColumnTransformer(transformers=[('onehot', onehot, categorical_features)], remainder='passthrough')
X_encoded = ct.fit_transform(X)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [51]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [52]:
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, random_state=42)

In [53]:
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)


In [54]:
print("=== Training Metrics ===")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1 Score:", train_f1)


=== Training Metrics ===
Accuracy: 0.545725
Precision: 0.5474956822107081
Recall: 0.5945412368589902
F1 Score: 0.5700494522395476


In [55]:
y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)


In [56]:
print("\n=== Test Metrics ===")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1 Score:", test_f1)



=== Test Metrics ===
Accuracy: 0.4935
Precision: 0.5081551286698079
Recall: 0.5439379243452959
F1 Score: 0.5254380211749273


In [57]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))



Confusion Matrix:
 [[2131 2714]
 [2351 2804]]

Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.44      0.46      4845
           1       0.51      0.54      0.53      5155

    accuracy                           0.49     10000
   macro avg       0.49      0.49      0.49     10000
weighted avg       0.49      0.49      0.49     10000



In [59]:
y_prob = model.predict_proba(X_test)[:, 1]
print("\nPredicted Flood Probabilities:\n", y_prob)


Predicted Flood Probabilities:
 [0.54475584 0.41308981 0.54358758 ... 0.57344932 0.46971628 0.49516055]


In [60]:
print(df['Flood Occurred'].value_counts())


Flood Occurred
1    25416
0    24584
Name: count, dtype: int64
